In [33]:
using Calculus;
using LinearAlgebra;
using Plots;
using Statistics;

#calibrate

nx  = 1;   # backward variables
ny  = 1;   # static variables
ne  = 1;   # shocks

rhoxx = -0.139;
etax  = 0.0348;
sxx   = etax/sqrt(1-rhoxx*rhoxx);

EP=[ 
   0.950 -1.500 0.0348 -0.139;
   0.500 -1.500 0.0348 -0.139;
   0.990 -1.500 0.0348 -0.139;
   0.950 -10.00 0.0348 -0.139;
   0.950  0.100 0.0348 -0.139;
   0.950  0.500 0.0348 -0.139;
   0.950 -1.500 0.0010 -0.139;
   0.950 -1.500 0.1000 -0.139;
   0.950 -1.500 sxx      0;
   0.950 -1.500 sxx*sqrt(0.75) 0.5;
   0.950 -1.500 sxx*sqrt(0.19) 0.9];


In [42]:
#define vectors
E1 = zeros(size(EP,1),3);
Ei = zeros(size(EP,1),3);
#iterate
#k=1
for k=1:size(EP,1);
beta  = EP[k,1];
theta = EP[k,2];
xb    = 0.0179;
rhox  = EP[k,4];
eta   = EP[k,3];
ETA   = [eta];
s2    =  1;
sx    = eta/sqrt(1-rhox*rhox);
ys    = beta*exp.(theta*xb)/(1-beta*exp.(theta*xb));
param =[beta theta rhox xb];

# steady state

devx  = 5;
nbx   = 200;
xmin  = xb-devx*sx;
xmax  = xb+devx*sx;
xgrid = collect(LinRange(xmin,xmax,nbx));

xs    = [xb ys xb ys];
xs    = vec(xs)

# True rule
nb=1000;
ri=rhox;
ri2=rhox;
ai= theta*xb+(theta*theta*eta*eta)/2;
bi= theta*rhox;
bti=beta;
tsol=bti.*exp.(ai.+bi.*(xgrid.-xb));
#i=1
for i=2:nb;
   ri   = rhox*ri;
   ri2  = rhox*rhox*ri2;
   ai   = theta*xb*i+(theta*theta*eta*eta)*(i-2*rhox*(1-ri)/(1-rhox)+rhox*rhox*(1-ri2)/(1-rhox*rhox))/(2*(1-rhox)*(1-rhox));
   bi   = theta*rhox*(1-ri)/(1-rhox);  
   bti  = bti*beta;
   tsol = tsol.+bti.*exp.(ai.+bi.*(xgrid.-xb));
end

# Jacobian matrix 
#using Calculus

#define the model
# variables (leads and lags)
#xp = xx(1);
#yp = xx(2);
#x  = xx(3);
#y  = xx(4);

#Jacobian
gr1 = Calculus.gradient(x -> x[1]-rhox*x[3]-(1-rhox)*xb, xs);
gr2 = Calculus.gradient(x -> x[4]-beta*exp.(theta*x[1])*(1+x[2]), xs);

#define the Jacobian
J =  [gr1'; gr2']

# Hessian
h1 = hessian(x -> x[1]-rhox*x[3]-(1-rhox)*xb, xs);
h2 = hessian(x -> x[4]-beta*exp.(theta*x[1])*(1+x[2]), xs);
h1=vec(h1);
h2=vec(h2);
#define the Hessian
H =  [h1 h2]

Gs,Hs,Gx,Hx,Gxx,Hxx,Gss,Hss = solve(xs,J,H,ETA,nx,ny,ne);

#Gx=vec(Gx); Hx=vec(Hx); Gxx=vec(Gxx);Hxx=vec(Hxx);


asol1 =Gs.+(Gx.*(xgrid.-xb));
asol2 =Gs.+0.5*Gss.+Gx.*(xgrid.-xb)+0.5*Gxx.*(xgrid.-xb).*(xgrid.-xb);
asoleq=Gs.+Gx.*(xgrid.-xb)+0.5*Gxx.*(xgrid.-xb).*(xgrid.-xb);

Err1=abs.((tsol.-asol1)./tsol);
Err2=abs.((tsol.-asol2)./tsol);
Erreq=abs.((tsol.-asoleq)./tsol);

Error = [Err1 Erreq Err2]

E1[k,:]=100*mean(Error, dims=1)
Ei[k,:]=100*maximum([Err1 Erreq Err2],dims=1);
end;

println("E1= ",E1)
println("Ei= ",Ei)


E1= [1.4414264268310046 1.4239103760589515 0.026882145392951116; 0.2537100781528589 0.2338327032065603 0.00409534114854602; 2.941371019700642 2.9242975054156437 0.0833374270203406; 23.77190811807874 23.134846118166134 4.577722746242568; 0.009866026974069833 0.00978763160570045 1.0597300883819018e-5; 0.2864661069012735 0.2845153346806518 0.0016351336140472655; 0.0011913776817333566 0.0011767099585567467 5.504757567490338e-7; 11.81999795625725 11.69012566841869 1.283447816334747; 1.8469234131761738 1.8469234131761738 0.03285082405474338; 5.914793764696314 4.9135985491185235 0.7099955008487492; 57.5111636572472 31.812725641680395 36.833636375534574]
Ei= [1.4774406044801596 1.4240816067732678 0.06419288202889661; 0.2944381500823517 0.23427716667008577 0.008691088728832303; 2.9764873253512913 2.9244445140083064 0.1736997313706534; 25.377376594104994 23.176421726839852 8.38804751911821; 0.010020908299387406 0.00978767363222938 2.1647823709263627e-5; 0.2903657362366111 0.2845213411289516 0.00

In [4]:
#sort eigenvalues functions
#qzswitch procedure

function reorder(s,t,q,z);
nsize = size(s,1);
i = 1;

while i<=nsize-1;

   if 1+abs(t[i,i]*s[i+1,i+1])>1+abs(s[i,i]*t[i+1,i+1]);    
      A=s; B=t; Q=q; Z=z;   

      s,t,q,z = qzswitch(i,A,B,Q,Z);
              
   i=i+1;
   end;         
end;            
return s,t,q,z;  
end;

function qzswitch(i,A,B,Q,Z);
    a = A[i,i]; d = B[i,i]; b =A[i,i+1]; e = B[i,i+1]; c = A[i+1,i+1]; f = B[i+1,i+1]; 

      wz = [c*e-f*b, (c*d-f*a)'];
      xy = [(b*d-e*a)', (c*d-f*a)'];
      n = sqrt(wz'*wz);
      m = sqrt(xy'*xy);

      if n == 0
      return
      else
    wz = \(n,wz);
    xy = \(m,xy); 
    wz = [wz[1] wz[2]; -wz[2] wz[1]]
    xy = [xy[1] xy[2]; -xy[2] xy[1]]
    A[i:i+1,:] = xy*A[i:i+1,:];
    B[i:i+1,:] = xy*B[i:i+1,:];
    A[:,i:i+1] = A[:,i:i+1]*wz;
    B[:,i:i+1] = B[:,i:i+1]*wz;
    Z[:,i:i+1] = Z[:,i:i+1]*wz;
    Q[i:i+1,:] = xy*Q[i:i+1,:];
       end;

   if ~(i==1);i = i-2;end   
return A,B,Q,Z;    
end; 

In [5]:
#Solve the model

function solve(xs,J,H,ETA,nx,ny,ne);

Hs=xs[1:nx];
Gs=xs[nx+1:nx+ny];

# Order 1

J0=J[:,1:nx+ny];
J1=-J[:,nx+ny+1:end];

r = schur(J0,J1)

s = r.S
t = r.T
q = r.Q  # So now q*a*z = s and q*b*z = t
z = r.Z
    
s,t,q,z=reorder(s,t,q,z)
    
z21 = z[nx+1:end,1:nx];
z11 = z[1:nx,1:nx];

if rank(z11)<nx;
 error("Invertibility condition violated")
end

z11i = \(z11,Matrix{Float64}(I, nx, nx));
s11 = s[1:nx,1:nx];
t11 = t[1:nx,1:nx];

#if abs(t[nx,nx])>abs(s[nx,nx]) | abs(t[nx+1,nx+1])<abs(s[nx+1,nx+1]);
#warning("Wrong number of stable eigenvalues");
#end 
    
#compute Gx and Hx
dyn = \(s11,t11);
Gx = z21*z11i;
Hx = z11*dyn*z11i;

tol=1e-6;
if maximum(maximum(imag(Gx)))<tol;
  Gx=real(Gx);
end
if maximum(maximum(imag(Gx)))<tol;
  Hx=real(Hx);
end;    
    
    
# Computes Gxx and Hxx

Zx = [Hx;Gx*Hx;Matrix{Float64}(I, nx, nx);Gx];
Jxp= J[:,1:nx];
Jyp= J[:,nx+1:nx+ny];
Jx = J[:,nx+ny+1:2*nx+ny];
Jy = J[:,2*nx+ny+1:2*(nx+ny)];
XX1 = [kron((Jxp+Jyp*Gx),Matrix{Float64}(I, nx*nx, nx*nx)) kron(Jyp,kron(Hx',Hx'))+kron(Jy,Matrix{Float64}(I, nx*nx, nx*nx))];
XX0 = -kron(Zx',Zx')*H;
XX0 = vec(XX0);
HGXX= \(XX1,XX0);
Hxx = HGXX[1:nx*nx*nx];
if maximum(maximum(imag(Hxx)))<tol
   Hxx=real(Hxx);
end
Gxx = HGXX[nx*nx*nx+1:end];
if maximum(maximum(imag(Gxx)))<tol
   Gxx=real(Gxx);
end;    

# Computes Gss and Hss

SS0= 0;
for i=1:ne;
    Zs    =[ETA[:,i];Gx*ETA[:,i];zeros(nx,1);zeros(ny,1)];
    TEMP0 = -kron(Zs',Zs')*H;
    TEMP0 = vec(TEMP0);
    TEMP1 = -Jyp*kron(Matrix{Float64}(I, ny, ny),kron(ETA[:,i]',ETA[:,i]'))*Gxx;
    TEMP1 = vec(TEMP1);
    SS0   = SS0.+TEMP0+TEMP1;
end

SS1  = [Jxp+Jyp*Gx (Jyp+Jy)];
HGSS = \(SS1,SS0);
Hss  = HGSS[1:nx];
Gss  = HGSS[nx+1:nx+ny];
if maximum(maximum(imag(Hss)))<tol
   Hss=real(Hss);
end
if maximum(maximum(imag(Gss)))<tol
   Gss=real(Gss);
end

Hxx = reshape(Hxx,nx*nx,nx)';
Gxx = reshape(Gxx,nx*nx,ny)';    
    
return Gs,Hs,Gx,Hx,Gxx,Hxx,Gss,Hss;
end;